In [ ]:
import pickle
import time
import datetime
import os.path
import matplotlib.pyplot as plt
import numpy as np
import random

f = open('history.dat', 'rb')
history_arr = pickle.load(f)


In [ ]:
def make_chart_arr():
    
    tm = []
    val = []
    
    for v in history_arr:
        open =  float(v[1])
        high =  float(v[2])
        low =   float(v[3])
        close = float(v[4])
        
        start_time = v[0]
        end_time = v[6]
        
        up_rotated = True
        if(len(val)>=4):
            up_rotated = open <= val[-4]
        
        tm.append(start_time)
        val.append(open)

        line_points = 5
        
        if up_rotated:
            for j in range(1,line_points):
                val.append(open+(high-open)*j/line_points)
                
            val.append(high)
            
            for j in range(1,line_points):
                val.append(high+(low-high)*j/line_points)
                
            val.append(low)
        
            for j in range(1,line_points):
                val.append(low+(close-low)*j/line_points)
        else:
            for j in range(1,line_points):
                val.append(open+(low-open)*j/line_points)
                
            val.append(low)
            
            for j in range(1,line_points):
                val.append(low+(high-low)*j/line_points)
                
            val.append(high)
        
            for j in range(1,line_points):
                val.append(high+(close-high)*j/line_points)
        
        for j in range(1,line_points*3):
            tm.append(int(start_time+(end_time-start_time)*j/(line_points*3)) )

        tm.append(end_time)
        val.append(close)
    
    return [tm,val]






In [ ]:
CHART_PER_DAY = 24*60

day_arr = []

def make_day_arr():
    global day_arr
    
    day_arr = []
    d = []
    
    for v in history_arr:
        open =  float(v[1])
        high =  float(v[2])
        low =   float(v[3])
        close = float(v[4])
        
        start_time = v[0]
        start_time = int(start_time/(CHART_PER_DAY*1000))*CHART_PER_DAY*1000;
        
        if len(d) != 0 and start_time > d[0]:
            day_arr.append(d)
            d = []
            
        if len(d) == 0:
            d = [start_time,open,high,low,close]
        else:
            d[4] = close
            d[2] = max(d[2],high)
            d[3] = min(d[3],low)


In [ ]:
def findDateIndex(date):
    unix_time = int(datetime.datetime.timestamp(date)*1000)
    index = 0
    for v in history_arr:
        if v[0]>=unix_time:
            break
        index += 1;
    return index
    
#from_index = findDateIndex(datetime.datetime(2022, 6, 14, 0, 0,0))
#from_index = findDateIndex(datetime.datetime(2022, 7, 1, 0, 0,0))
#to_index = findDateIndex(datetime.datetime(2022, 6, 14, 0, 0,0))

#history_arr = history_arr[from_index:to_index]
#history_arr = history_arr[from_index:]

make_day_arr()
x,y = make_chart_arr()

min_price = min(y)
max_price = max(y)

date_time = datetime.datetime.fromtimestamp(x[0]/1000)
print("from=", date_time.strftime('%Y-%m-%d %H:%M:%S'))

date_time = datetime.datetime.fromtimestamp(x[-1]/1000)
print("to=", date_time.strftime('%Y-%m-%d %H:%M:%S'))

print("min_price=", min_price)
print("max_price=", max_price)


In [ ]:
day_x=np.arange(0,len(x),dtype=float)
IDX_IN_DAY = CHART_PER_DAY*16
day_x /= IDX_IN_DAY


In [ ]:
%matplotlib notebook
plt.plot(day_x,y)
#plt.xlim(20, 190)
#plt.ylim(0, 80000)
plt.show()


In [ ]:
buyMoneyAmmount = 100
buyRatio = 18300/19000
flatRatio = 18700/18300

stableDelay=1*60

def isTimeToBuy(idx,price):
    if(idx<stableDelay):
        return False
    
    mh2 = history_arr[idx-stableDelay]
    price_high=float(mh2[2])

    if price/price_high < buyRatio:
        return False
    
    return True



In [ ]:
buy_points = []
sell_points = []
loss_points = []

SEARCH_BUY = 1
BUY = 2
SEARCH_SELL = 3
    
sellRatio = 18050/18000
stopLossRatio = 17500/18000
sellMinutesTimeout = 2*60

money = 100
btc = 0

last_time_buy = 0
last_buy_price = 0

state = SEARCH_BUY

order_points = []
order_money = []

next_random = int(random.random()*16)

for i in range(4,len(y)):
    price = y[i]
    minute = int(i/16)
    
    if state == SEARCH_BUY:
        if (i%16)==next_random and isTimeToBuy(minute,price):
            next_random = int(random.random()*16)
            state = BUY
    elif state == BUY:
        if money<= 100:
            btc = money/price
            money = 0
        else:
            btc = 100/price
            money -= 100
        
        last_time_buy = minute
        last_buy_price = price
        order_points.append(day_x[i]);
        order_money.append(money+btc*price);
        buy_points.append(day_x[i])
        print(f'{minute}: buy price={price}')
        state  = SEARCH_SELL
    elif state == SEARCH_SELL:
        doSell = False
        
        sellPrice = last_time_buy
        lossPrice = last_time_buy
        
        if minute<last_time_buy + sellMinutesTimeout:
            dt=1.0-(minute-last_time_buy)/sellMinutesTimeout
            sellPrice = last_buy_price+(last_buy_price*sellRatio-last_buy_price)*dt
            lossPrice = last_buy_price+(last_buy_price*stopLossRatio-last_buy_price)*dt
        
        if price>= sellPrice:
            print(f'{minute}: sell price={price}')
            sell_points.append(day_x[i])
            doSell = True
        elif price<= lossPrice:
            print(f'{minute}: loss price={price}')
            print(f'dt={dt} last_buy_price={last_buy_price} sellPrice={sellPrice} lossPrice={lossPrice}')
            loss_points.append(day_x[i])
            doSell = True
            
        if doSell:
            money+=btc*price
            btc=0
            order_points.append(day_x[i]);
            order_money.append(money+btc*price);
            state  = SEARCH_BUY


order_points.append(day_x[-1]);
order_money.append(money+btc*price);

In [ ]:
print("money=",money)
print("state=",state)
print("buy_points:",len(buy_points),"sell_points:",len(sell_points),"loss_points:",len(loss_points))


In [ ]:
%matplotlib notebook
plt.plot(day_x,y)

for p in buy_points:
    plt.vlines(p,min_price,max_price,color='green')

for p in sell_points:
    plt.vlines(p,min_price,max_price,color='red')

for p in loss_points:
    plt.vlines(p,min_price,max_price,color='grey')

plt.show()

In [ ]:
fig, ax1 = plt.subplots()
color1 ='blue'
ax1.plot(day_x,y, color = color1)
ax1.set_ylabel('Price',color = color1);
ax1.tick_params(axis='y',color = color1);

color2 ='crimson'
ax2 = ax1.twinx();
ax2.plot(order_points,order_money, color = color2)
ax2.set_ylabel('Money', color = color2);
ax2.tick_params(axis='y',color = color2);


In [ ]:
print("min_money",min(order_money))
print("max_money",max(order_money))
print("money=",order_money[-1])